In [ ]:
!pip install camelot-py pandas
!pip install PyPDF2==2.9.0

In [ ]:
#pip install pdfplumber

In [ ]:
import pandas as pd
from transformers import pipeline
import camelot

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
def extract_tables_from_pdf(pdf_path):

    tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
    dataframes = [table.df for table in tables]
    return dataframes

def preprocess_table(table_df):

    # Drop empty columns and rows
    table_df = table_df.dropna(how='all', axis=1).dropna(how='all', axis=0)

    # Reset index
    table_df.columns = table_df.iloc[0]
    table_df = table_df[1:].reset_index(drop=True)

    return table_df

"""
def table_to_text(table_df):

    table_text = ""
    for index, row in table_df.iterrows():
        row_text = " , ".join(f"{col} {val}" for col, val in row.items())
        table_text += f"{row_text}\n"
    return table_text
"""

# For table 8
def table_to_text(table_df):

    table_text = ""
    for index, row in table_df.iterrows():
        # Get the label for each row
        label = row.iloc[0]

        # Create sentence for each column and its value, pass the first column
        for col, val in row.items():
            if col != label and val != "" and val != label:
              table_text += f"{label} in {col} is {val}.\n"

    return table_text


def preprocess_table(table_df, table_num):

      if table_num == 8 or 10:
        table_df = table_df.iloc[1:]
        table_df.columns = table_df.iloc[0]
        table_df = table_df.iloc[1:]

      return table_df


def financial_qa(document_text, question):

    qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")
    # qa_pipeline = pipeline("question-answering", model="allenai/numnet_plus")

    # if there is no answer, arrange in here

    result = qa_pipeline(question=question, context=document_text)['answer']

    return result

"""
def financial_qa_bert(document_text):

    model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3)
    tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

    nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

    results = nlp(document_text)

    return results
"""

In [ ]:
pdf_path = 'finqa_table.pdf'
tables = extract_tables_from_pdf(pdf_path)

In [ ]:
chosen_table = tables[10]

In [ ]:
chosen_table

In [ ]:
chosen_table = preprocess_table(chosen_table, 10)

In [ ]:
chosen_table

In [ ]:
# transposed_table = chosen_table.set_index(chosen_table.columns[0]).T

In [ ]:
table_text = table_to_text(chosen_table)
print("Table as Text:\n", table_text)

In [ ]:
question = "What is Non-GAAP net incomes in 2022?"
answer = financial_qa(table_text, question)

# answer = financial_qa_bert(table_text)

print(f"Answer: {answer}")
